In [1]:
import pandas as pd #Для работы с таблицами данных (дата фреймы)
import requests #Для запросов к серверу
import json #Для обработки ответов сервера
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import apimoex
import yfinance as yf
from IPython.display import display

In [2]:
import apimoex
import requests

def ticker(ticker_name, start, end):
    with requests.Session() as session:
        data = pd.DataFrame()
        new_data = pd.DataFrame(apimoex.get_market_history(session=session,
                                                           security=ticker_name,
                                                           start=start,
                                                           end=end))
        data = pd.concat([data, new_data], axis=0, join='outer')
    data['Names'] = [ticker_name] * data.shape[0]
    data = data.sort_values(by='TRADEDATE')
    data = pd.pivot_table(data, index='TRADEDATE', columns='Names', values='CLOSE')
    data.index = pd.to_datetime(data.index)
    data.index = data.index.date
    return(data)


In [3]:
all_companies = []
with open("D:/Jupyter notebook/files/НИР/Список всех компаний.txt", 'r') as file:
    for item in file:
        all_companies += [item]
file.close
all_companies = [string.strip() for string in all_companies]

In [4]:
all_companies = np.unique(all_companies)

In [5]:
len(all_companies)

1245

### Компании, которые меняли свой ticker

In [6]:
changeover = pd.read_excel('D:/Jupyter notebook/files/НИР/changeover.xlsx')

In [7]:
changeover[changeover.new_secid == 'SBER']

,action_date,old_secid,new_secid
112,2011-11-21,SBER03,SBER
368,2007-07-11,SBER,SBER
369,2007-07-11,SBER-001D,SBER


In [8]:
dict_comp = {company: [company] for company in all_companies}

In [9]:
for col1_val, col2_val in zip(changeover['old_secid'],changeover['new_secid']):
    # Check if the value in column 2 is a key in the dictionary
    if col2_val in dict_comp :
        # If it is, add the value from column 1 to that key's value in the dictionary
        dict_comp[col2_val] += [col1_val]
for key in dict_comp:
    dict_comp[key] = list(set(dict_comp[key]))

In [10]:
dict_comp

{'486-ME': ['486-ME'],
 'A-RM': ['A-RM'],
 'AA-RM': ['AA-RM'],
 'AAL-RM': ['AAL-RM'],
 'AAP-RM': ['AAP-RM'],
 'AAPL': ['AAPL'],
 'AAPL-RM': ['AAPL-RM'],
 'ABBN': ['ABBN'],
 'ABBV-RM': ['ABBV-RM'],
 'ABC-RM': ['ABC-RM'],
 'ABMD-RM': ['ABMD-RM'],
 'ABNB-RM': ['ABNB-RM'],
 'ABRD': ['ABRD'],
 'ABT-RM': ['ABT-RM'],
 'ACAD-RM': ['ACAD-RM'],
 'ACKO': ['ACKO'],
 'ACN-RM': ['ACN-RM'],
 'ADBE-RM': ['ADBE-RM'],
 'ADI-RM': ['ADI-RM'],
 'ADM-RM': ['ADM-RM'],
 'ADP-RM': ['ADP-RM'],
 'ADSK-RM': ['ADSK-RM'],
 'AEE-RM': ['AEE-RM'],
 'AEO-RM': ['AEO-RM'],
 'AEP-RM': ['AEP-RM'],
 'AER-RM': ['AER-RM'],
 'AES-RM': ['AES-RM'],
 'AESL': ['AESL'],
 'AFKS': ['AFKC', 'AFKS'],
 'AFL-RM': ['AFL-RM'],
 'AFLT': ['AFLT', 'RU0009062285'],
 'AGRE': ['AGRE'],
 'AIG-RM': ['AIG-RM'],
 'AIZ-RM': ['AIZ-RM'],
 'AJG-RM': ['AJG-RM'],
 'AKAM-RM': ['AKAM-RM'],
 'AKHA': ['AKHA-001D', 'AKHA'],
 'AKRN': ['AKRN'],
 'ALB-RM': ['ALB-RM'],
 'ALBK': ['ALBK'],
 'ALGN-RM': ['ALGN-RM'],
 'ALK-RM': ['ALK-RM'],
 'ALL-RM': ['ALL-RM'],
 'ALLY

In [15]:
all_shares = pd.DataFrame(index=pd.date_range('2000-03-31', '2023-04-06', freq='D'))

for company, tickers in dict_comp.items():
    df_1 = pd.DataFrame()
    for ticker_name in set(tickers):
        try:
            share = ticker(ticker_name, start='2000-01-01', end='2023-07-01')
            if len(share) == 0:
                continue
            else: 
                # Rename column to company name
                share = share.rename(columns={ticker_name: company})
                share.index = pd.to_datetime(share.index)
                share = share.sort_index()
                
                # Add to all_shares dataframe
                df_1 = pd.concat([df_1, share], axis=0)
                
                # Check for duplicates and replace NaN values
                df_1 = df_1[~df_1.index.duplicated(keep='last')]
                df_1 = df_1.fillna(method='ffill')
        except KeyError:
            print(f"Data not found for {ticker_name}")
            continue

    if len(df_1) != 0:
        start_date = pd.to_datetime('2000-01-01')
        end_date = df_1.index.min()

        if start_date < end_date :
            new_index = pd.date_range(start_date, end_date, freq='D')
            df = pd.DataFrame(index=new_index, columns=[company])
            df.drop(df.tail(1).index, inplace=True)
            df_1 = pd.concat([df, df_1], axis=0)

        start_date = df_1.index.max()
        end_date = pd.to_datetime('2023-04-11')

        if start_date < end_date:
            new_index = pd.date_range(start_date, end_date, freq='D')
            df = pd.DataFrame(index=new_index, columns=[company])
            df.drop(df.head(1).index, inplace=True)
            df_1 = pd.concat([df_1, df], axis=0)
        
        all_shares = pd.concat([all_shares, df_1[~df_1.index.duplicated(keep='first')][company]], axis=1)

#all_shares.index = all_shares.index.date


Data not found for 486-ME
Data not found for A-RM
Data not found for AA-RM
Data not found for AAL-RM
Data not found for AAP-RM
Data not found for AAPL
Data not found for AAPL-RM
Data not found for ABBV-RM
Data not found for ABC-RM
Data not found for ABMD-RM
Data not found for ABNB-RM
Data not found for ABT-RM
Data not found for ACAD-RM
Data not found for ACN-RM
Data not found for ADBE-RM
Data not found for ADI-RM
Data not found for ADM-RM
Data not found for ADP-RM
Data not found for ADSK-RM
Data not found for AEE-RM
Data not found for AEO-RM
Data not found for AEP-RM
Data not found for AER-RM
Data not found for AES-RM
Data not found for AFL-RM
Data not found for AIG-RM
Data not found for AIZ-RM
Data not found for AJG-RM
Data not found for AKAM-RM
Data not found for ALB-RM
Data not found for ALGN-RM
Data not found for ALK-RM
Data not found for ALL-RM
Data not found for ALLY-RM
Data not found for AMAT-RM
Data not found for AMC-RM
Data not found for AMD-RM
Data not found for AME-RM
Data n

Data not found for HPQ-RM
Data not found for HRB-RM
Data not found for HRL-RM
Data not found for HRTX-RM
Data not found for HSIC-RM
Data not found for HSY-RM
Data not found for HUM-RM
Data not found for HWM-RM
Data not found for HYLN-RM
Data not found for IBM-RM
Data not found for ICE-RM
Data not found for IDXX-RM
Data not found for IEX-RM
Data not found for IFF-RM
Data not found for IIVI-RM
Data not found for ILMN-RM
Data not found for INCY-RM
Data not found for INSG-RM
Data not found for INTC
Data not found for INTC-RM
Data not found for INTU-RM
Data not found for IONS-RM
Data not found for IOVA-RM
Data not found for IP-RM
Data not found for IPG-RM
Data not found for IPGP-RM
Data not found for IQV-RM
Data not found for IR-RM
Data not found for ISRG-RM
Data not found for IT-RM
Data not found for ITW-RM
Data not found for J-RM
Data not found for JBHT-RM
Data not found for JEF-RM
Data not found for JKHY-RM
Data not found for JNJ-RM
Data not found for JNPR-RM
Data not found for JPM-RM
Da

Data not found for TROW-RM
Data not found for TRV-RM
Data not found for TSCO-RM
Data not found for TSLA
Data not found for TSLA-RM
Data not found for TSN-RM
Data not found for TSP-RM
Data not found for TSVT-RM
Data not found for TTCF-RM
Data not found for TTD-RM
Data not found for TTWO-RM
Data not found for TUI1-RM
Data not found for TWLO-RM
Data not found for TWNK-RM
Data not found for TWOU-RM
Data not found for TWTR
Data not found for TWTR-RM
Data not found for TXN-RM
Data not found for TXT-RM
Data not found for TYL-RM
Data not found for U-RM
Data not found for UA-RM
Data not found for UAL-RM
Data not found for UBER-RM
Data not found for UHS-RM
Data not found for ULTA-RM
Data not found for UNH-RM
Data not found for UNM-RM
Data not found for UNP-RM
Data not found for UPS-RM
Data not found for UPWK-RM
Data not found for URBN-RM
Data not found for URI-RM
Data not found for USB-RM
Data not found for USFD-RM
Data not found for V
Data not found for V-RM
Data not found for VERT
Data not fou

In [16]:
all_shares

,ABBN,ABRD,ACKO,AESL,AFKS,AFLT,AKHA,AKRN,ALBK,ALNU,...,YRSB,YRSBP,YRSL,ZHIV,ZHIV-001D,ZILL,ZMZN,ZMZNP,ZOYA,ZVEZ
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,6.900,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-10,NaN,267.00,NaN,NaN,15.1370,35.540,NaN,18908.0,NaN,NaN,...,226.0,103.0,NaN,NaN,NaN,3855.0,NaN,NaN,NaN,5.940
2023-04-11,NaN,258.75,NaN,NaN,15.2880,34.080,NaN,18948.0,NaN,NaN,...,228.0,99.5,NaN,NaN,NaN,3750.0,NaN,NaN,NaN,6.095
2023-04-12,NaN,259.00,NaN,NaN,14.3655,34.490,NaN,19116.0,NaN,NaN,...,230.0,103.5,NaN,NaN,NaN,3735.0,NaN,NaN,NaN,5.730
2023-04-13,NaN,250.00,NaN,NaN,14.0345,35.585,NaN,19120.0,NaN,NaN,...,226.0,103.0,NaN,NaN,NaN,3830.0,NaN,NaN,NaN,5.750


In [68]:
#Есть всякие странности с акциями. К примеру акции Сбера, которые выдает очень странные данные.
#Дело в том, что в данных Сбербанка есть странности
#http://iss.moex.com/iss/history/engines/stock/markets/shares/securities/SBER-001D
#https://iss.moex.com/iss/history/engines/stock/markets/shares/securities/changeover

all_shares.SBER.loc['2007-07-01':'2009-01-01']

2007-07-01         NaN
2007-07-02    100490.0
2007-07-03    100005.0
2007-07-04    100100.0
2007-07-05    101000.0
                ...   
2008-12-28         NaN
2008-12-29        22.0
2008-12-30        23.0
2008-12-31        23.0
2009-01-01         NaN
Freq: D, Name: SBER, Length: 551, dtype: float64

In [25]:
#Можно посмотреть на акции с наименьшим кол-вом Nan
all_shares.isna().sum().sort_values()

AFLT         2698
SNGS         2698
LKOH         2698
SNGSP        2699
SIBN         2700
             ... 
KUBE-003D    8502
USYN         8503
HMBO         8504
ERCO02       8504
UCSS-001D    8504
Length: 472, dtype: int64

### Find label of first non-null value

In [24]:
# Find label of first non-null value
first_non_null_label = (all_shares.isnull().sum(axis=1) == len(all_shares.columns)).idxmin()
first_non_null_label

Timestamp('2000-01-05 00:00:00', freq='D')

In [22]:
print(all_shares.loc[first_non_null_label])

ABBN     NaN
ABRD     NaN
ACKO     NaN
AESL     NaN
AFKS     NaN
        ... 
ZILL     NaN
ZMZN     0.0
ZMZNP    0.0
ZOYA     NaN
ZVEZ     NaN
Name: 2000-01-05 00:00:00, Length: 472, dtype: float64
